# Ch. 1 - Setup

First we import the required dependencies.

In [205]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd
from pandas.plotting import scatter_matrix
import urllib.request

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# seaborn
import seaborn as sns

Then we load the data provided on Aulis from github to our project directory.

In [206]:
DATASETS_URL = "https://github.com/TurboThommy/ml4ds-end-to-end-project/blob/main/data/"
DATASET_DIR = "datasets"

# File names
TRAINING_DATA_FILENAME = "Vehicles_export_prices_scaled_train_eng.xlsx"
TEST_DATA_FILENAME = "Vehicles_export_prices_scaled_stud_test_eng.xlsx"

def fetch_file_from_url(src_url, dest_path, file_name):
    if not os.path.isdir(dest_path):
        os.makedirs(dest_path)
    file_url = src_url + file_name + "?raw=true"
    file_path = os.path.join(dest_path, file_name)
    urllib.request.urlretrieve(file_url, file_path)

fetch_file_from_url(DATASETS_URL, DATASET_DIR, TRAINING_DATA_FILENAME)

Load the training data set and print the first few rows.

In [207]:
TRAINING_DATA_PATH = os.path.join("datasets", TRAINING_DATA_FILENAME)

def load_data(data_path):
    return pd.read_excel(data_path)

training_data_df = load_data(TRAINING_DATA_PATH)
training_data_df.head()

Print information about the loaded data frame.

In [208]:
training_data_df.info(verbose=True)

Sort all columns in the data frame alphabetically and print them with their corresponding data types.

In [209]:
columns = []
for col in training_data_df.columns:
    columns.append([col.title(), training_data_df[col.title().upper()].dtype])
columns.sort(key=lambda x: x[0])
columns

Create a correlation matrix for the training data.

In [210]:
corr_matrix = training_data_df.select_dtypes(include=["number"]).corr()
corr_matrix["LAID_UP_TIME"].sort_values(ascending=False)

In [211]:
attributes = ["LAID_UP_TIME", "SCALED_INVENTURAL_VALUE"]
scatter_matrix(training_data_df[attributes])
plt.show()

In [212]:
# attr = training_data_df.select_dtypes(include=["number"])
# attr.head()
# scatter_matrix(attr)
# plt.show()
# filtered_df = training_data_df.loc[training_data_df['AT_LOCATION_SINCE'] > 1000000]
# filtered_df.head()

import sklearn.model_selection


filterd_df, _ = sklearn.model_selection.train_test_split(training_data_df, train_size=0.1, random_state=42)

attr = [
    "LAID_UP_TIME",
    "SCALED_INVENTURAL_VALUE",
    "AT_LOCATION_SINCE",
    "VEHICLE_MODEL_ID",
    "NUMBER_AXLE",
    "IS_USED_CAR",
    "CURB_WEIGHT",
    "NUMBER_SEATS",
    "CONSTRUCTION_MONTH",
    "NUMBER_DOORS",
    "YEAR_CONSTRUCTION",
    "HORSEPOWER",
    "OPTICAL_CONDITION",
    "KW",
    "OPERATING_HOURS",
    "OPERATION_HOURS_SALE",
    "OPERATING_HOURS_SALES",
    "SCALED_CURRENT_VALUE",
    "SCALED_TOTAL_SALE_PRICE",
    "SCALED_TOTAL_SALES_PRICE_BASIS",
    "SCALED_GUIDE_PRICE",
    "NUMBER_OWNERS",
    "CCM",
    "MAX_TRAILOR_LOAD",
    "MILEAGE",
    "PERMITTED_TOTAL_WEIGHT",
    "NUMBER_ENGINE_CYLINDER",
    "MILAGE_SALE",
    "MILAGE_SALES",
    "LEASING_MILAGE",
    "PURCHASE_MILAGE",
    "COMPANY",
    "SCALED_REPORT_VALUE",
    # "MILAGE_IN_FIELD",
    # "CERTIFICATE_TYPE_DATE",
    # "FRONT_TIRES_CONDITION",
    # "REAR_TIRES_CONDITION",
    # "REPAIR_RKZ",
    # "TECHNICAL_CONDITION",
    # # "PAINT_TYPE", => only nulls
    # "PURCHASE_OPERATION_HOURS",
    # "SCALED_VALUATION_PRICE"
]
scatter_matrix(filterd_df[attr], figsize=(100, 100))
plt.show()

In [213]:
training_data_df["SCALED_INVENTURAL_VALUE"].describe()

In [214]:
print(len(training_data_df["MODEL_CODE"].unique()))
print(training_data_df["COLOR_CODE"].isnull().sum())

In [215]:
corr_matrix = training_data_df.select_dtypes(include=["number"]).corr(method="pearson")
plt.figure(figsize=(30, 30))
sns.heatmap(corr_matrix, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

## Prepare Data

### Keep columns of interest

In [216]:
def can_be_mapped(src: str, dest: str) -> bool:
    uni_map = list(training_data_df.groupby(src)[dest].nunique())
    return max(uni_map) == 1

def can_be_mapped_bidir(a: str, b: str) -> bool:
    return can_be_mapped(a, b) and can_be_mapped(b, a)

def count_na(col: str, df: pd.DataFrame) -> int:
    return df[col].isna().sum()

# print(count_na("COLOR_CODE", training_data_df))
# print(can_be_mapped("CUSTOMER_SALE_GROUP_NAME", "CUSTOMER_SALE_GROUP2_NAME"))
# print(training_data_df["COLOR_CODE"].nunique())

def print_stats(col: str) -> None:
    print(f'NA: {count_na(col, training_data_df)}')
    print(f'Unique: {training_data_df[col].nunique()}')
    print(f'Top Cats:\n{training_data_df[col].value_counts().head(10)}')
    plt.hist(training_data_df[col].value_counts(), bins=50)

print_stats("CUSTOMER_SALE_GROUP2_NAME")

In [217]:
columns_of_interest = [
    "RPAKREP_VEHICLE_HKEY",
    "COMPANY",
    "OFFICE",
    # "OFFICE_MAIN_BRAND", # could be used for an additional feature (comparison to MANUFACTURER_SHORT)
    # "CHASSIS_NUMBER", # irrelevant as it is a unique identifier for the vehicle
    "MANUFACTURER_SHORT",
    # "MANUFACTURER", # contains more null values then MANUFACTURER_SHORT
    "VEHICLE_GROUP",
    # "VEHICLE_TYPE", # contains too many unique entries (>13,700)
    # "MODEL_CODE", # contains many null values and too many unique ones
    # "VARIANT", # mostly null values
    "MILEAGE",
    # "OPERATING_HOURS", # contains just 6 unique values, mostly 0s
    # "MILAGE_IN_FIELD", # contains just 1s and nulls, mostly 1s
    "MILAGE_SALES", # the difference between MILEAGE and MILAGE_SALES might be more relevant
    # "OPERATING_HOURS_SALES", # contains just 5 unique values, mostly 0s
    # "RIM_KEY", # contains mostly nulls
    # "COLOR_CODE", # contains only 30% not nulls
    # "COLOR_CODE_NAME", # contains mostly 'Nicht definiert'
    "COLOR",
    # "COLOR_TYPE", # contains mostly 'N'
    # "UPHOLSTERY_CODE", # contains only 30% not nulls
    "UPHOLSTERY",
    # "UPHOLSTERY_CODE_ALT", # contains mostly 'N'
    # "CERTIFICATE_TYPE", # mostly nulls
    # "CERTIFICATE_TYPE_DATE", # mostly 0s
    # "FACTORY_NUMBER", # mostly nulls
    # "ENGINE_ID", # many nulls, more like an id
    "ENGINE_TYPE",
    # "ENGINE_ID_ALT", # 60% nulls
    # "TRANSMISSION", # 80% nulls
    "TRANSMISSION_TYPE",
    # "TRANSMISSION_ID", # 80% nulls
    # "TRANSMISSION_SHORT", # mostly mappable to TRANSMISSION_NAME
    "TRANSMISSION_NAME",
    # "RIMS", # mostly nulls
    # "FRONT_TIRES", # mostly nulls
    # "FRONT_TIRES_CONDITION", # mostly 0.00
    # "REAR_TIRES", # mostly nulls
    # "REAR_TIRES_CONDITION", # mostly 0.00
    "NUMBER_DOORS",
    "NUMBER_SEATS",
    # "PERMITTED_TOTAL_WEIGHT", # 85% 0s
    # "MAX_TRAILOR_LOAD", # 95% 0s
    "CURB_WEIGHT",
    "YEAR_CONSTRUCTION",
    # "CONSTRUCTION_MONTH", # might not be relevant
    # "NUMBER_AXLE", # mostly 0s
    "NUMBER_ENGINE_CYLINDER",
    # "REPAIR_RKZ", # only 0s and nulls
    # "OPTICAL_CONDITION", # only 0s and nulls
    # "TECHNICAL_CONDITION", # only 0s and nulls
    "ACCIDENT_VEHICLE",
    # "COMMISSION_NUMBER", # too many uniques
    "HORSEPOWER",
    # "KW", # horsepower egould be enough
    "CCM",
    "NUMBER_OWNERS",
    # "IS_USED_CAR", # can be mapped from NUMBER_OWNERS
    # "LEASING_CONTRACT_DATE", # mostly nulls
    # "LEASING_START", # mostly nulls
    # "LEASING_END", # mostly nulls
    # "LEASING_MILAGE", # mostly 0s
    # "PAINT_TYPE", # only nulls
    # "FINANCING_TYPE", # FINANCING_TYPE_NAME is already enough
    "FINANCING_TYPE_NAME",
    # "KAT_VEHICLE", # one class is overrepresented
    # "FUEL_TYPE", # FUEL_TYPE_NAME is already enough
    "FUEL_TYPE_NAME",
    # "DRIVE_TYPE", # DRIVE_TYPE_NAME is already enough
    "DRIVE_TYPE_NAME",
    # "VEHICLE_MODEL_ID", # VEHICLE_MODEL_ID_NAME is already enough
    "VEHICLE_MODEL_ID_NAME",
    # "COMMISSION_TYPE", # COMMISSION_TYPE_NAME is already enough
    "COMMISSION_TYPE_NAME",
    # "DEMONSTRATION_STATUS", # mostly nulls
    "PURCHASE_DATE",
    # "PURCHASE_BOOKING_DATE", # not relevant due to PURCHASE_DATE
    # "PURCHASE_MILAGE", # the difference between PURCHASE_MILAGE and MILAGE_SALES might be more relevant
    # "PURCHASE_OPERATION_HOURS", # only 0s and nulls
    # "PRICE_LIST", # mostly nulls
    # "DAY_OF_REGISTRATION", # mostly nulls and 'N'
    # "AT_LOCATION_SINCE", # mostly 0s
    "LAID_UP_TIME",
    # "SOLD_CUSTOMER_ID", # too many different classes
    # "SOLD_INVOICE_COSTUMER_ID", # too many different classes
    # "MILAGE_SALE", # mostly identical to MILAGE_SALES
    # "OPERATION_HOURS_SALE", mostly 0s
    # "SOLD_INVOICE_COSTUMER_ID2", # too many different classes
    "CUSTOMER_TYPE",
    # "CUSTOMER_GROUP", # CUSTOMER_SALE_GROUP2_NAME is already enough
    # "CUSTOMER_GROUP_NAME", # CUSTOMER_SALE_GROUP2_NAME is already enough
    # "CUSTOMER_FEATURE", # CUSTOMER_SALE_GROUP2_NAME is already enough
    # "CUSTOMER_FEATURE_NAME", # CUSTOMER_SALE_GROUP2_NAME is already enough
    # "SALE_CUSTOMER_ID2", # too many different classes
    # "CUSTOMER_SALE_GROUP", # CUSTOMER_SALE_GROUP2_NAME is already enough
    # "CUSTOMER_SALE_GROUP_NAME", # CUSTOMER_SALE_GROUP2_NAME is already enough
    # "CUSTOMER_SALE_GROUP2", # CUSTOMER_SALE_GROUP2_NAME is already enough
    "CUSTOMER_SALE_GROUP2_NAME",
    "SCALED_CURRENT_VALUE",
    "SCALED_INVENTURAL_VALUE",
    "SCALED_REPORT_VALUE",
    # "SCALED_VALUATION_PRICE", # only contains 0.00
    "SCALED_GUIDE_PRICE",
    "SCALED_TOTAL_SALES_PRICE_BASIS",
    "SCALED_TOTAL_SALE_PRICE"
]

training_data_prepared_df = training_data_df[columns_of_interest].copy()
training_data_prepared_df.head()

### Convert dtypes

In [218]:
def map_values_to_na(col: str, values: list[any]):
    training_data_prepared_df[col] = training_data_prepared_df[col].map(lambda x: np.nan if x in values else x)

# RPAKREP_VEHICLE_HKEY
# COMPANY
training_data_prepared_df["COMPANY"] = training_data_prepared_df["COMPANY"].astype("Int64").astype("object")
# OFFICE
training_data_prepared_df["OFFICE"] = training_data_prepared_df["OFFICE"].astype("object")
# MANUFACTURER_SHORT
training_data_prepared_df["MANUFACTURER_SHORT"] = training_data_prepared_df["MANUFACTURER_SHORT"].astype("object")
map_values_to_na("MANUFACTURER_SHORT", ["*"])
# VEHICLE_GROUP
training_data_prepared_df["VEHICLE_GROUP"] = training_data_prepared_df["VEHICLE_GROUP"].astype("object")
map_values_to_na("VEHICLE_GROUP", ["*"])
# MILEAGE
# COLOR
training_data_prepared_df["COLOR"] = training_data_prepared_df["COLOR"].astype("object")
map_values_to_na("COLOR", ["-"])
# UPHOLSTERY
training_data_prepared_df["UPHOLSTERY"] = training_data_prepared_df["UPHOLSTERY"].astype("object")
map_values_to_na("UPHOLSTERY", ["0", "1", "-, -", "#NAME?"])
# ENGINE_TYPE
training_data_prepared_df["ENGINE_TYPE"] = training_data_prepared_df["ENGINE_TYPE"].astype("object")
map_values_to_na("ENGINE_TYPE", ["BITTE MOT-ID PFLEGEN", "-", "*", ".", "#NAME?"])
# TRANSMISSION_TYPE
training_data_prepared_df["TRANSMISSION_TYPE"] = training_data_prepared_df["TRANSMISSION_TYPE"].astype("object")
# TRANSMISSION_NAME
training_data_prepared_df["TRANSMISSION_NAME"] = training_data_prepared_df["TRANSMISSION_NAME"].astype("object")
map_values_to_na("TRANSMISSION_NAME", ["Nicht definiert"])
# NUMBER_DOORS
map_values_to_na("NUMBER_DOORS", [0.0])
# NUMBER_SEATS
map_values_to_na("NUMBER_SEATS", [-1.0, 0.0])
# CURB_WEIGHT
map_values_to_na("CURB_WEIGHT", [0.0])
# YEAR_CONSTRUCTION
training_data_prepared_df["YEAR_CONSTRUCTION"] = training_data_prepared_df["YEAR_CONSTRUCTION"].map(lambda x: np.nan if x < 1900 or x > 2024 else x)
# NUMBER_ENGINE_CYLINDER
training_data_prepared_df["NUMBER_ENGINE_CYLINDER"] = training_data_prepared_df.apply(lambda x: np.nan if x["NUMBER_ENGINE_CYLINDER"] < 1 and x["FUEL_TYPE_NAME"] == "Elektro" else x, axis=1)["NUMBER_ENGINE_CYLINDER"]
# ACCIDENT_VEHICLE
training_data_prepared_df["ACCIDENT_VEHICLE"] = training_data_prepared_df["ACCIDENT_VEHICLE"].map({"N": False, 1: True}, na_action="ignore")
training_data_prepared_df["ACCIDENT_VEHICLE"] = training_data_prepared_df["ACCIDENT_VEHICLE"].astype("boolean")
# HORSEPOWER
map_values_to_na("HORSEPOWER", [0.0])
# CCM
map_values_to_na("CCM", [0.0])
# NUMBER_OWNERS
# FINANCING_TYPE_NAME
training_data_prepared_df["FINANCING_TYPE_NAME"] = training_data_prepared_df["FINANCING_TYPE_NAME"].astype("object")
map_values_to_na("FINANCING_TYPE_NAME", ["Nicht definiert"])
# FUEL_TYPE_NAME
training_data_prepared_df["FUEL_TYPE_NAME"] = training_data_prepared_df["FUEL_TYPE_NAME"].astype("object")
# DRIVE_TYPE_NAME
training_data_prepared_df["DRIVE_TYPE_NAME"] = training_data_prepared_df["DRIVE_TYPE_NAME"].astype("object")
map_values_to_na("DRIVE_TYPE_NAME", ["Nicht definiert"])
# VEHICLE_MODEL_ID_NAME
training_data_prepared_df["VEHICLE_MODEL_ID_NAME"] = training_data_prepared_df["VEHICLE_MODEL_ID_NAME"].astype("object")
map_values_to_na("VEHICLE_MODEL_ID_NAME", ["Keine Zuteilung"])
# COMMISSION_TYPE_NAME
training_data_prepared_df["COMMISSION_TYPE_NAME"] = training_data_prepared_df["COMMISSION_TYPE_NAME"].astype("object")
# PURCHASE_DATE
# LAID_UP_TIME
# CUSTOMER_TYPE
training_data_prepared_df["CUSTOMER_TYPE"] = training_data_prepared_df["CUSTOMER_TYPE"].astype("object")
# CUSTOMER_SALE_GROUP2_NAME
training_data_prepared_df["CUSTOMER_SALE_GROUP2_NAME"] = training_data_prepared_df["CUSTOMER_SALE_GROUP2_NAME"].astype("object")
# SCALED_CURRENT_VALUE
map_values_to_na("SCALED_CURRENT_VALUE", [training_data_prepared_df["SCALED_CURRENT_VALUE"].value_counts().nlargest(1).keys()[0]])
# SCALED_INVENTURAL_VALUE
map_values_to_na("SCALED_INVENTURAL_VALUE", [0.0])
# SCALED_REPORT_VALUE
# SCALED_VALUATION_PRICE
# SCALED_GUIDE_PRICE
map_values_to_na("SCALED_GUIDE_PRICE", [training_data_prepared_df["SCALED_GUIDE_PRICE"].value_counts().nlargest(1).keys()[0]])
# SCALED_TOTAL_SALES_PRICE_BASIS
# SCALED_TOTAL_SALE_PRICE

### Remove certain rows and fill certain cells

In [219]:
# ToDo
# Leasing Cars?
# 67 -> often seen as null count

# Remove all rows where LAID_UP_TIME is na
print(len(training_data_prepared_df))
training_data_prepared_df.dropna(subset=["LAID_UP_TIME"], inplace=True)
print(len(training_data_prepared_df))

In [220]:
training_data_prepared_df.isna().sum()

In [221]:
training_data_prepared_df.isna().sum(axis=1).hist()

In [222]:
print(len(training_data_prepared_df))
training_data_prepared_df.dropna(thresh=24, inplace=True)
print(len(training_data_prepared_df))
print(training_data_prepared_df.isna().sum())

In [223]:
training_data_fully_cleared_df = training_data_prepared_df.dropna().copy()

# OFFICE: drop rows
training_data_prepared_df.dropna(subset=["OFFICE"], inplace=True)
# MANUFACTURER_SHORT: drop rows
training_data_prepared_df.dropna(subset=["MANUFACTURER_SHORT"], inplace=True)
# VEHICLE_GROUP: fille with new catetory
training_data_prepared_df["VEHICLE_GROUP"] = training_data_prepared_df["VEHICLE_GROUP"].fillna("Unknown")
# COLOR: fille with new catetory
training_data_prepared_df["COLOR"] = training_data_prepared_df["COLOR"].fillna("Unknown")
# UPHOLSTERY: fille with new catetory
training_data_prepared_df["UPHOLSTERY"] = training_data_prepared_df["UPHOLSTERY"].fillna("Unknown")
# ENGINE_TYPE:
# TRANSMISSION_TYPE:
# TRANSMISSION_NAME:
# NUMBER_DOORS:
# NUMBER_SEATS:
# CURB_WEIGHT:
# YEAR_CONSTRUCTION:
# NUMBER_ENGINE_CYLINDER:
# ACCIDENT_VEHICLE: fill with False
training_data_prepared_df["ACCIDENT_VEHICLE"] = training_data_prepared_df["ACCIDENT_VEHICLE"].fillna(False)
# HORSEPOWER:
# CCM:
# FINANCING_TYPE_NAME: fille with new catetory
training_data_prepared_df["FINANCING_TYPE_NAME"] = training_data_prepared_df["FINANCING_TYPE_NAME"].fillna("Unknown")
# DRIVE_TYPE_NAME:
# VEHICLE_MODEL_ID_NAME:
# PURCHASE_DATE:
# CUSTOMER_TYPE:
# CUSTOMER_SALE_GROUP2_NAME:

training_data_fully_cleared_df.isna().sum()

### Add derived features to the dataframe

In [224]:
# OFFICE_MAIN_BRAND = MANUFACTURER_SHORT
# MILEAGE - MILAGE_SALES (PURCHASE_MILAGE) (MILAGE_SALE)
# milage_diff = training_data_fully_cleared_df["MILAGE_SALES"] - training_data_fully_cleared_df["MILEAGE"]
# training_data_fully_cleared_df["MILEAGE_DIFF"] = milage_diff

### Investigate features

In [225]:
corr_matrix = training_data_fully_cleared_df.select_dtypes(include=["number"]).corr()
corr_matrix["LAID_UP_TIME"].sort_values(ascending=False)

In [226]:
scatter_matrix(training_data_fully_cleared_df.select_dtypes(include=["number"]), figsize=[60, 60])
plt.show()

In [227]:
training_data_fully_cleared_df["OFFICE"].value_counts().plot(kind='bar', figsize=(15, 5))
plt.show()

In [231]:
cat_features = training_data_fully_cleared_df.select_dtypes(include=["object"]).columns
plt.figure(figsize=(15, 300))
for i, feature in enumerate(cat_features):
    if len(training_data_fully_cleared_df[feature].value_counts()) <= 50:
        plt.subplot(len(cat_features), 1, i + 1)
        plt.scatter(training_data_fully_cleared_df[feature].astype('str'), training_data_fully_cleared_df["LAID_UP_TIME"], alpha=0.25, linewidths=0)
        plt.xticks(rotation=90)
        plt.xlabel(feature)
        plt.ylabel("Laid Up Time")
plt.show()

In [229]:
corr_matrix = training_data_fully_cleared_df.select_dtypes(include=["number"]).corr(method="pearson")
plt.figure(figsize=(15, 15))
sns.heatmap(corr_matrix, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()
